In [1]:
from pathlib import Path
from typing import Tuple, Union, List, Dict, Iterable, Optional

import torch
import numpy as np
from tqdm.notebook import tqdm

from decomposer import Decomposer, DecomposerConfig
from recomposer import Recomposer, RecomposerConfig
from helpers import *

Loading vocabulary from /home/webson/Research/congressional_adversary/results/3bins/pretrained/init.pt
Vocab size = 138,444


In [2]:
# base_dir = Path('../../results/news/validation')
base_dir = Path('../../results/3bins/deno space/').resolve()
deno_space = load_decomposers_en_masse(base_dir, patterns='*/epoch1.pt')
# deno_space.update(load_decomposers_en_masse(base_dir, patterns='*/epoch3.pt'))
print(deno_space.keys())

Loading /home/webson/Research/congressional_adversary/results/3bins/deno space/-3c L1/epoch1.pt

dict_keys(['-3c L1/epoch1.pt'])


/home/webson/Research/congressional_adversary/congressional_env/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'decomposer.Decomposer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [3]:
from evaluations.word_similarity.all_wordsim import mean_delta, read_plain_text, compare
import math

SELECT_MODEL = deno_space['-3c L1/epoch1.pt']

def convert(embed):
    word_vecs: Dict[str, np.array] = {}
    for word_id, vector in enumerate(embed):
        vector /= math.sqrt((vector ** 2).sum() + 1e-6)
        word_vecs[ITW[word_id]] = vector
    return word_vecs

def cf_WS(m1, m2):
    compare(convert(m1), convert(m2))

def cf(query):
    GD(query)
    print('Pretrained:')
    nearest_neighbors(query, PE)
    print('Our model:')
    nearest_neighbors(query, SELECT_MODEL)
    
cf_WS(SELECT_MODEL, PE)

             Dataset             Rho       Reference      Delta >.02
    EN-MTurk-771.txt          8.12%         52.84%         -44.73%
  EN-RW-STANFORD.txt          9.38%         45.33%         -35.96%
   EN-SIMLEX-999.txt          1.58%         32.92%         -31.34%
    EN-MTurk-287.txt         15.80%         59.11%         -43.31%
       EN-YP-130.txt         21.58%         50.72%         -29.14%
   EN-WS-353-SIM.txt         -0.23%         68.51%         -68.75%
    EN-MEN-TR-3k.txt          9.64%         52.49%         -42.85%
   EN-WS-353-REL.txt          7.61%         44.70%         -37.09%
     EN-VERB-143.txt         10.08%         29.87%         -19.79%
   EN-WS-353-ALL.txt         -0.93%         58.16%         -59.08%
 EN-SimVerb-3500.txt         -1.72%         26.25%         -27.97%
Mean Delta = -40.0006%


In [4]:
GD('aca')

aca	1.0, 0.0, 0.0, 	1, 0, 0, 


In [5]:
cf('estate_tax')
cf('death_tax')

estate_tax	1.0, 0.0, 0.0, 	0, 0, 0, 
Pretrained:
0.8058	death_tax
0.7939	inheritance_tax
0.7891	alternative_minimum
0.7830	surtax
0.7819	richest_americans
0.7713	top_tax
0.7479	individual_tax
0.7476	trillion_tax
0.7456	marriage_penalty
0.7389	top_rate

Our model:
0.7116	death_tax
0.7092	alternative_minimum
0.6906	richest_americans
0.6893	surtax
0.6808	individual_tax
0.6787	top_tax
0.6610	inheritance_tax
0.6530	trillion_tax
0.6425	top_rate
0.6412	marginal_tax

death_tax	0.0, 1.0, 0.0, 	0, 0, 0, 
Pretrained:
0.8740	inheritance_tax
0.8266	surtax
0.8179	estate_taxes
0.8133	wealthy_tax
0.8133	obamacare_taxes
0.8108	alternative_minimum
0.8090	cuts_expire
0.8088	personal_exemptions
0.8077	raises_taxes
0.8068	marriage_penalty

Our model:
0.7772	inheritance_tax
0.7238	surtax
0.7116	estate_tax
0.7022	alternative_minimum
0.6990	obamacare_taxes
0.6825	increasing_taxes
0.6767	wealthy_tax
0.6754	repeals
0.6735	cuts_expire
0.6696	estate_taxes



In [6]:
cf('undocumented_workers')
cf('illegal_aliens')
cf('chain_migration')

undocumented_workers	0.2054, 0.7946, 0.0, 	0, 0, 0, 
Pretrained:
0.7386	unauthorized_workers
0.7354	work_visas
0.7099	hire_illegal
0.7078	million_undocumented
0.7017	temporary_work
0.7002	young_undocumented
0.6967	million_unauthorized
0.6966	work_authorization
0.6951	young_illegal
0.6937	called_dreamers

Our model:
0.6423	million_undocumented
0.6102	work_visas
0.6074	million_illegal
0.5970	work_permits
0.5889	unauthorized_immigrants
0.5882	undocumented_immigrant
0.5794	legal_immigrants
0.5767	immigrants_would
0.5731	immigration_agents
0.5695	u.s._illegally

illegal_aliens	0.0, 0.0, 1.0, 	0, 0, 0, 
Pretrained:
0.7453	illegal_immigrants
0.7422	illegals
0.7150	undocumented_immigrants
0.7089	criminal_illegal
0.6989	criminal_aliens
0.6969	illegal_immigrant
0.6916	million_illegal
0.6883	young_illegal
0.6855	unauthorized_immigrants
0.6716	million_immigrants

Our model:
0.9356	illegal_immigrants
0.9290	undocumented_immigrants
0.9101	immigrants
0.9093	deportation
0.9066	illegals
0.9059	undocume

In [7]:
cf('obamacare')
cf('aca')
cf('public_option')
cf('single_payer')
cf('socialized_medicine')

obamacare	1.0, 0.0, 0.0, 	0, 0, 0, 
Pretrained:
0.8302	aca
0.8285	affordable_care
0.7863	health_law
0.7632	act_aca
0.7479	obamacare_insurance
0.7416	obamacare_would
0.7394	patient_protection
0.7364	republican_plan
0.7360	healthcare_law
0.7340	care_law

Our model:
0.9872	health_care
0.9856	affordable_care
0.9838	republicans
0.9824	democrats
0.9823	bill
0.9822	gop
0.9821	act
0.9820	plan
0.9818	obama
0.9817	americans

aca	1.0, 0.0, 0.0, 	1, 0, 0, 
Pretrained:
0.8302	obamacare
0.8112	affordable_care
0.7640	individual_mandate
0.7463	health_law
0.7154	act_aca
0.7030	patient_protection
0.6975	obamacare_insurance
0.6948	healthcare_law
0.6799	buy_health
0.6786	obamacare_would

Our model:
0.9624	affordable_care
0.9550	obamacare
0.9432	health_insurance
0.9301	insurance
0.9258	health_care
0.9257	medicaid
0.9236	medicare
0.9227	repeal
0.9218	coverage
0.9181	insurance_companies

public_option	0.0655, 0.9345, 0.0, 	0, 1, 0, 
Pretrained:
0.7152	medicare_buy
0.6861	public_plan
0.6798	health_reform
0.66

In [8]:
cf('leftists')
cf('antifa')

leftists	0.0, 0.0, 1.0, 	0, 0, 0, 
Pretrained:
0.6805	called_liberals
0.6750	liberals
0.6546	america_hating
0.6530	sjws
0.6524	liberals_progressives
0.6519	left_wingers
0.6471	hate_mongers
0.6435	marxists
0.6434	radical_leftists
0.6414	upper_caste

Our model:
0.9496	liberals
0.9422	conservatives
0.9375	progressives
0.9361	activists
0.9354	liberal
0.9336	folks
0.9334	democrats
0.9332	supporters
0.9323	republicans
0.9319	right_wing

antifa	0.0, 0.0, 1.0, 	0, 0, 0, 
Pretrained:
0.7640	anti-fascist
0.7229	alt_left
0.7210	deray_mckesson
0.7154	blacklives_matter
0.7036	anti-fascists
0.7026	matter_protesters
0.6984	anti-trump_protesters
0.6881	black_bloc
0.6795	white_supremacists
0.6775	jason_kessler

Our model:
0.7369	neo-nazis
0.7344	white_supremacists
0.7305	kkk
0.7222	anarchists
0.7210	alt_right
0.7177	matter_movement
0.7136	black_lives
0.7058	fascists
0.7008	neo-nazi
0.7007	charlottesville



In [9]:
cf('guns')

guns	0.8041, 0.1959, 0.0, 	0, 0, 0, 
Pretrained:
0.7178	firearms
0.6305	gun_enthusiasts
0.6286	semi-automatic_weapons
0.6231	ammo
0.6207	handguns
0.6139	gun_laws
0.6128	buy_guns
0.6080	assault_rifles
0.6066	firearm
0.6060	semiautomatic_weapons

Our model:
0.9766	weapons
0.9716	more
0.9716	them
0.9715	are
0.9715	people
0.9715	these
0.9715	they
0.9715	[PAD]
0.9715	on
0.9714	out



In [10]:
cf('trade')

trade	0.0, 0.0, 1.0, 	0, 0, 0, 
Pretrained:
0.5864	education_trade
0.5812	infrastructure_corporate
0.5559	actual_trade
0.5520	yen_carry
0.5515	trade_could
0.5504	investment_agreements
0.5465	corporate_regulation
0.5437	investment_flows
0.5435	u.s._goods
0.5382	corporate_governance

Our model:
0.9899	more
0.9899	united_states
0.9898	and
0.9897	both
0.9897	on
0.9896	as
0.9896	china
0.9896	of
0.9896	<UNK>
0.9895	but



In [11]:
cf('crooked_hillary')
cf('lyin_ted')
cf('little_marco')

crooked_hillary	0.8952, 0.1048, 0.0, 	0, 0, 0, 
Pretrained:
0.7717	lyin
0.7677	lyin_ted
0.7644	lying_media
0.7625	nasty_woman
0.7605	little_marco
0.7530	candidate_ever
0.7529	attack_hillary
0.7513	attacking_hillary
0.7499	fucking_moron
0.7467	el_trumpo

Our model:
0.6315	lyin
0.6172	el_trumpo
0.6086	nasty_woman
0.5972	vast_right
0.5955	like_hillary
0.5918	lyin_ted
0.5891	attacking_hillary
0.5889	pathological_liar
0.5839	nominee_hillary
0.5812	wing_conspiracy

lyin_ted	0.1624, 0.8376, 0.0, 	0, 1, 0, 
Pretrained:
0.8914	little_marco
0.8196	lyin
0.8062	small_hands
0.7978	ted_cruz.
0.7936	trump_calling
0.7881	real_donald
0.7807	cruz_wife
0.7804	snit
0.7803	still_trump
0.7776	typical_politician

Our model:
0.7804	little_marco
0.6398	lyin
0.6257	cruz_wife
0.6229	small_hands
0.6213	trump_calling
0.6147	heidi_cruz
0.6076	still_trump
0.5957	called_donald
0.5952	skidmore_flickr
0.5918	crooked_hillary

little_marco	0.7079, 0.2921, 0.0, 	0, 0, 0, 
Pretrained:
0.8914	lyin_ted
0.8575	lyin
0.8505	ted

In [12]:
cf('lgbt')

lgbt	0.9584, 0.0416, 0.0, 	0, 0, 0, 
Pretrained:
0.7207	gay_lesbian
0.6757	bisexual
0.6623	transgender_community
0.6552	lgbt_movement
0.6512	lgbt_community
0.6449	lesbian_rights
0.6414	lesbian_gay
0.6403	transgender
0.6371	transgender_issues
0.6366	bisexual_transgender

Our model:
0.9352	gay
0.9218	discrimination
0.9209	transgender
0.9197	civil_rights
0.9181	lesbian
0.9148	gays
0.9145	religious
0.9131	advocates
0.9129	women
0.9127	black

